In [4]:
# UNCOMMENT THESE TO INSTALL THE REQUIRED LIBRARIES

# pip install pandas
# pip install tensorflow
# pip install scikit-learn
# pip install pymysql
# pip install sshtunnel

In [3]:
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
import pymysql
import sshtunnel
import tensorflowjs as tfjs


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

print("Numpy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("Tensorflow version:", tf.__version__)
print("Sklearn version:", sklearn.__version__)
print("Pymysql version:", pymysql.__version__)
print("Sshtunnel version:", sshtunnel.__version__)

D:\ThongLai\OneDrive - University of Huddersfield\Team Project\workIt\AI\source\Gym_Recommender_venv\Lib\site-packages\tensorflowjs\read_weights.py:28: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  np.uint8, np.uint16, np.object, np.bool]


AttributeError: module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

### Reading data from a SQL-SERVER

In [5]:
# Set up SSH tunnel
server = sshtunnel.SSHTunnelForwarder(
    ('selene.hud.ac.uk', 22),
    ssh_username='workit',
    ssh_password='umbra(despair>Quartz218',
    remote_bind_address=('localhost', 3306)
)
server.start()

# Defining the connection
connection = pymysql.connect(host='localhost', port=server.local_bind_port, user='workit', password='umbra(despair>Quartz218', database='workit')

In [6]:
# DATAPATH = "../input/megaGymDataset.csv"

# Load the dataset
dataset = pd.read_sql_query("SELECT * FROM `megaGymDataset`", connection)
dataset.head()

,ID,Title,Desc,Type,BodyPart,Equipment,Level,Rating
0,0,Partner plank band row,The partner plank band row is an abdominal exe...,Strength,Abdominals,Bands,Intermediate,0.0
1,1,Banded crunch isometric hold,The banded crunch isometric hold is an exercis...,Strength,Abdominals,Bands,Intermediate,4.5
2,2,FYR Banded Plank Jack,The banded plank jack is a variation on the pl...,Strength,Abdominals,Bands,Intermediate,4.5
3,3,Banded crunch,The banded crunch is an exercise targeting the...,Strength,Abdominals,Bands,Intermediate,4.5
4,4,Crunch,The crunch is a popular core exercise targetin...,Strength,Abdominals,Bands,Intermediate,4.5


In [7]:
server.stop()

### Training from dataset

In [8]:
def labelEncodersScaler(data):
    label_encoders = {}
    for feature in ["Type", "BodyPart", "Equipment", "Level"]:
        label_encoders[feature] = LabelEncoder()
        data[feature] = label_encoders[feature].fit_transform(data[feature])

    X = data[["Type", "BodyPart", "Equipment", "Level"]]
    y = data["ID"]
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Standardize the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)

    return label_encoders, scaler

In [9]:
def train_model(data):
    label_encoders, scaler = labelEncodersScaler(data)
    
    X = data[["Type", "BodyPart", "Equipment", "Level"]]
    y = data["ID"]
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Build the ANN model using TensorFlow
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(len(data["ID"].unique()), activation='softmax')  # Use the unique number of IDs as the output dimension
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

    return model

In [17]:
model = train_model(dataset.copy())
model.save("ann_gym_recommender.h5")

Epoch 1/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0000e+00 - loss: 7.9918 - val_accuracy: 0.0000e+00 - val_loss: 8.0169
Epoch 2/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0050 - loss: 7.9111 - val_accuracy: 0.0000e+00 - val_loss: 8.5342
Epoch 3/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0043 - loss: 7.0978 - val_accuracy: 0.0000e+00 - val_loss: 10.1052
Epoch 4/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0103 - loss: 6.2777 - val_accuracy: 0.0000e+00 - val_loss: 12.3136
Epoch 5/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0138 - loss: 5.7491 - val_accuracy: 0.0000e+00 - val_loss: 13.9870
Epoch 6/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0190 - loss: 5.4018 - val_accuracy: 0.0000e+00 - val_loss: 15.5034
Epoch 7/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0371 - loss: 5.0747 - val_accuracy: 0.0000e+00 - val_loss: 16.8478
Epoch 8/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0308 - loss: 4.92

In [19]:
!tensorflowjs_converter --input_format=keras "ann_gym_recommender.h5" /tmp/tfjs_model

'tensorflowjs_converter' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
tfjs.converters.save_keras_model(model, "hi")

TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates